<a href="https://colab.research.google.com/github/diego2017003/projeto-MNIST_machine_learning/blob/development/HyperparameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install idx2numpy==1.2.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%%capture
!pip install wandb==0.10.17

In [ ]:
!pip install tensorflow

In [4]:
# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

# Wandb
import wandb
from wandb.keras import WandbCallback

# Bibliotecas Auxiliares
import idx2numpy
import gzip
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
import pytz
import IPython

In [5]:
!wandb login --relogin

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
run = wandb.init(project="mnist_deep_classification",save_code=True,job_type='train')
dataset_train = 'preprocessed_x_train.npz'
artifact_train = run.use_artifact(f"mnist_deep_classification/{dataset_train}:latest",type='npz')
artifact_train.file()

wandb: Currently logged in as: diego25rm (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


'./artifacts/preprocessed_x_train.npz:v0/preprocessed_x_train.npz'

In [7]:
train_x = np.load(artifact_train.file())['imgs']

In [8]:
y_dataset_train = 'train-labels-idx1-ubyte.gz'
y_artifact_train= run.use_artifact(f"mnist_deep_classification/{y_dataset_train}:latest",type='gzip')
y_artifact_train.file()

'./artifacts/train-labels-idx1-ubyte.gz:v0/train-labels-idx1-ubyte.gz'

In [9]:
with gzip.open(y_artifact_train.file(), 'r') as f:
    train_y = idx2numpy.convert_from_file(f)

In [10]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.fit(train_x, train_y, epochs=15)

Epoch 1/15
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2623 - accuracy: 0.9255
Epoch 2/15
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1149 - accuracy: 0.9664
Epoch 3/15
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0792 - accuracy: 0.9763
Epoch 4/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0590 - accuracy: 0.9822
Epoch 5/15
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0457 - accuracy: 0.9854
Epoch 6/15
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0349 - accuracy: 0.9889
Epoch 7/15
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0282 - accuracy: 0.9911
Epoch 8/15
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0242 - accuracy: 0.9924
Epoch 9/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0182 - accuracy: 0.9947
Epoch 10/15
1875/1875 [==============================] - 5s 3ms/step - loss: 0.015

In [13]:
dataset_test = 'preprocessed_x_test.npz'
artifact_test = run.use_artifact(f"mnist_deep_classification/{dataset_test}:latest",type='npz')
artifact_test.file()
test_x = np.load(artifact_test.file())['imgs']

In [14]:
y_dataset_test = 't10k-labels-idx1-ubyte.gz'
y_artifact_test= run.use_artifact(f"mnist_deep_classification/{y_dataset_test}:latest",type='gzip')
y_artifact_test.file()
with gzip.open(y_artifact_test.file(), 'r') as f:
    test_y = idx2numpy.convert_from_file(f)

In [15]:
test_loss, test_acc = model.evaluate(test_x, test_y, verbose=2)

313/313 - 1s - loss: 0.0987 - accuracy: 0.9772 - 570ms/epoch - 2ms/step


**Hyperparameter Tuning**

**Monitoring a neural network**

In [16]:
# Default values for hyperparameters
defaults = dict(layer_1 = 8,
                layer_2 = 8,
                learn_rate = 0.001,
                batch_size = 32,
                epoch = 500)

wandb.init(project="week06", config= defaults, name="week06_run_01")
config = wandb.config

wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [17]:
%%wandb
# Add WandbCallback() to the fit function
model = keras.Sequential([
                                keras.layers.Flatten(input_shape=(28, 28)),
                                keras.layers.Dense(config.layer_1, activation='relu', dtype='float64'),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(config.layer_2, activation='relu', dtype='float64'),
                                keras.layers.Dense(10, activation='softmax', dtype='float64')
                                ])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) 

model.fit(x=train_x,
          y=train_y,
          batch_size=config.batch_size,
          epochs=config.epoch,
          validation_data=(test_x,test_y),
          callbacks=[WandbCallback(log_weights=True)],
          verbose=1)

Epoch 1/500
1875/1875 [==============================] - 6s 3ms/step - loss: 0.8351 - accuracy: 0.7480 - val_loss: 0.4452 - val_accuracy: 0.8720
Epoch 2/500
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4592 - accuracy: 0.8665 - val_loss: 0.3666 - val_accuracy: 0.8997
Epoch 3/500
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4082 - accuracy: 0.8842 - val_loss: 0.3471 - val_accuracy: 0.9063
Epoch 4/500
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3853 - accuracy: 0.8910 - val_loss: 0.3533 - val_accuracy: 0.9019
Epoch 5/500
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3664 - accuracy: 0.8953 - val_loss: 0.3262 - val_accuracy: 0.9089
Epoch 6/500
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3513 - accuracy: 0.9000 - val_loss: 0.3184 - val_accuracy: 0.9105
Epoch 7/500
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3341 - accuracy: 0.9039 - val_loss: 0.3047 - val_ac

**Sweeps**

In [ ]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    defaults = dict(layer_1 = 100,
                layer_2 = 100,
                batch_size = 32,
                epoch = 20)

    wandb.init(project="mnist_deep_classification",save_code=True,job_type='train', config= defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Instantiate a simple classification model
    model = keras.Sequential([
                                keras.layers.Flatten(input_shape=(28, 28)),
                                keras.layers.Dense(config.layer_1, activation='relu', dtype='float64'),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(config.layer_2, activation='relu', dtype='float64'),
                                keras.layers.Dense(10, activation='softmax', dtype='float64')
                                ])


    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) 

    model.fit(train_x, train_y, batch_size=config.batch_size,
              epochs=config.epoch,
              validation_data=(test_x,test_y)
              )

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'binary_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'layer_1': {
            'max': 32,
            'min': 8,
            'distribution': 'int_uniform',
        },
        'layer_2': {
            'max': 32,
            'min': 8,
            'distribution': 'int_uniform',
        },
        'learn_rate': {
            'min': -4,
            'max': -2,
            'distribution': 'log_uniform',  
        },
        'epoch': {
            'values': [300,400,600]
        },
        'batch_size': {
            'values': [32,64]
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="diegao", project="week06")

In [ ]:
wandb.agent(sweep_id = sweep_id, function=train,count=20)